In [ ]:
import numpy as np
import sys
import itertools as itertools
sys.path.append("..")
sys.path.append("../..")

import matplotlib.pyplot as plt
from python.Heisenberg import XXZ_model, nn_Heisenberg_model, XY_model
from python.DMRG import DMRG
from python.Zippers import MPO_to_Hamiltonian, contract_MPS
from python.Canonical_Form import get_Neumann_entropy

In [2]:
n_sites = 10

ZZ_coupling = 1.0
XY_coupling = 1.0
magnetic_field = 0.0

Hamiltonian = XXZ_model(
    n_sites=n_sites, ZZ_coupling=ZZ_coupling,
    XY_coupling=XY_coupling, magnetic_field=magnetic_field,
)
MPO_transposed = [ham.transpose(2, 3, 0, 1) for ham in Hamiltonian]

In [ ]:
"""
Get exact matrix (exponential barrier)
"""

matrix = MPO_to_Hamiltonian(MPO_transposed)
eigvals, _ = np.linalg.eigh(matrix)
eigvals[:10]

In [4]:
NKeep = 20
NSweep = 10
Krylov_bases = 5 #* 5 is usually enough
Lanczos_cutoff = 1e-2
iterative_diag = True
two_site = True
verbose = True

In [ ]:
ground_energies, ground_times, ground_MPS = DMRG(
    Hamiltonian = Hamiltonian,
    NKeep = NKeep,
    NSweep = NSweep,
    Krylov_bases = Krylov_bases,
    Lanczos_cutoff = Lanczos_cutoff,
    iterative_diag = iterative_diag,
    two_site = two_site,
    verbose = verbose,
)

eigvals[0]

L=10 | NKeep=20 | NSweep=10 | iterative=True | two_site=True | Krylov_bases=5 | Lanczos_cutoff=0.01
iter=0 | energy=-7.1778373 | time=0.0s
iter=1 | energy=-7.1805011 | time=0.296s
iter=2 | energy=-7.1805011 | time=0.409s
iter=3 | energy=-7.1805011 | time=0.567s
iter=4 | energy=-7.1805011 | time=0.739s
iter=5 | energy=-7.1805011 | time=0.901s
iter=6 | energy=-7.1805011 | time=1.02s
iter=7 | energy=-7.1805011 | time=1.23s
iter=8 | energy=-7.1805011 | time=1.38s
iter=9 | energy=-7.1805011 | time=1.52s
iter=10 | energy=-7.1805011 | time=1.62s


In [ ]:
excited_energies, excited_times, excited_MPS = DMRG(
    Hamiltonian = Hamiltonian,
    NKeep = NKeep,
    NSweep = NSweep,
    Krylov_bases = Krylov_bases,
    Lanczos_cutoff = Lanczos_cutoff,
    iterative_diag = False,
    orthogonal_to_list_of_MPS=[ground_MPS],
    two_site = two_site,
    verbose = verbose,
)

eigvals[1]

L=10 | NKeep=20 | NSweep=10 | iterative=False | two_site=True | Krylov_bases=5 | Lanczos_cutoff=0.01
iter=0 | energy=-0.064555886 | time=0.0s
iter=1 | energy=-6.7185007 | time=1.24s
iter=2 | energy=-6.7185007 | time=2.03s
iter=3 | energy=-6.7185007 | time=3.05s
iter=4 | energy=-6.7185007 | time=3.52s
iter=5 | energy=-6.7185007 | time=4.12s
iter=6 | energy=-6.7185007 | time=4.56s
iter=7 | energy=-6.7185007 | time=5.3s
iter=8 | energy=-6.7185007 | time=6.15s
iter=9 | energy=-6.7185007 | time=7.31s
iter=10 | energy=-6.7185007 | time=8.51s


In [8]:
n_sites = 10

J1 = 1.0
J2 = 1.0

Hamiltonian = nn_Heisenberg_model(
    n_sites=n_sites, J1=J1, J2=J2,
)
MPO_transposed = [ham.transpose(2, 3, 0, 1) for ham in Hamiltonian]

In [ ]:
"""
Get exact matrix (exponential barrier)
"""

matrix = MPO_to_Hamiltonian(MPO_transposed)
eigvals, _ = np.linalg.eigh(matrix)
eigvals[:10]

array([-2.25, -2.25, -1.75, -1.75, -1.75, -1.75, -1.75, -1.75, -1.75,
       -1.75])

In [13]:
NKeep = 20
NSweep = 10
Krylov_bases = 5 #* 5 is usually enough
Lanczos_cutoff = 1e-2
iterative_diag = True
two_site = True
verbose = True

In [14]:
ground_energies, ground_times, ground_MPS = DMRG(
    Hamiltonian = Hamiltonian,
    NKeep = NKeep,
    NSweep = NSweep,
    Krylov_bases = Krylov_bases,
    Lanczos_cutoff = Lanczos_cutoff,
    iterative_diag = iterative_diag,
    two_site = two_site,
    verbose = verbose,
)

eigvals[0]

L=10 | NKeep=20 | NSweep=10 | iterative=True | two_site=True | Krylov_bases=5 | Lanczos_cutoff=0.01
iter=0 | energy=-2.25 | time=0.0s
iter=1 | energy=-2.25 | time=0.0263s
iter=2 | energy=-2.25 | time=0.0519s
iter=3 | energy=-2.25 | time=0.0776s
iter=4 | energy=-2.25 | time=0.0979s
iter=5 | energy=-2.25 | time=0.117s
iter=6 | energy=-2.25 | time=0.136s
iter=7 | energy=-2.25 | time=0.157s
iter=8 | energy=-2.25 | time=0.176s
iter=9 | energy=-2.25 | time=0.196s
iter=10 | energy=-2.25 | time=0.215s


-2.25

In [15]:
excited_energies, excited_times, excited_MPS = DMRG(
    Hamiltonian = Hamiltonian,
    NKeep = NKeep,
    NSweep = NSweep,
    Krylov_bases = Krylov_bases,
    Lanczos_cutoff = Lanczos_cutoff,
    iterative_diag = False,
    orthogonal_to_list_of_MPS=[ground_MPS],
    two_site = two_site,
    verbose = verbose,
)

eigvals[1]

L=10 | NKeep=20 | NSweep=10 | iterative=False | two_site=True | Krylov_bases=5 | Lanczos_cutoff=0.01
iter=0 | energy=0.050230742 | time=0.0s
iter=1 | energy=-2.2499805 | time=0.208s
iter=2 | energy=-2.2499805 | time=0.352s
iter=3 | energy=-2.2499805 | time=0.462s
iter=4 | energy=-2.2499805 | time=0.674s
iter=5 | energy=-2.2499805 | time=0.799s
iter=6 | energy=-2.2499805 | time=0.871s
iter=7 | energy=-2.2499805 | time=0.997s
iter=8 | energy=-2.2499805 | time=1.06s
iter=9 | energy=-2.2499805 | time=1.13s
iter=10 | energy=-2.2499805 | time=1.19s


-2.25

In [16]:
n_sites = 10

J = -1.0
Gamma = 1.0

Hamiltonian = XY_model(
    n_sites=n_sites, J=J, Gamma=Gamma,
)
MPO_transposed = [ham.transpose(2, 3, 0, 1) for ham in Hamiltonian]

In [ ]:
"""
Get exact matrix (exponential barrier)
"""

matrix = MPO_to_Hamiltonian(MPO_transposed)
eigvals, _ = np.linalg.eigh(matrix)
eigvals[:10]

array([-5.28482978, -4.75150298, -4.66134396, -4.53573529, -4.39498427,
       -4.25373069, -4.12801717, -4.12188325, -4.00629712, -4.00240849])

In [18]:
ground_energies, ground_times, ground_MPS = DMRG(
    Hamiltonian = Hamiltonian,
    NKeep = NKeep,
    NSweep = NSweep,
    Krylov_bases = Krylov_bases,
    Lanczos_cutoff = Lanczos_cutoff,
    iterative_diag = iterative_diag,
    two_site = two_site,
    verbose = verbose,
)

eigvals[0]

L=10 | NKeep=20 | NSweep=10 | iterative=True | two_site=True | Krylov_bases=5 | Lanczos_cutoff=0.01
iter=0 | energy=-5.2770168 | time=0.0s
iter=1 | energy=-5.2848298 | time=0.113s
iter=2 | energy=-5.2848298 | time=0.25s
iter=3 | energy=-5.2848298 | time=0.283s
iter=4 | energy=-5.2848298 | time=0.32s
iter=5 | energy=-5.2848298 | time=0.37s
iter=6 | energy=-5.2848298 | time=0.413s
iter=7 | energy=-5.2848298 | time=0.455s
iter=8 | energy=-5.2848298 | time=0.487s
iter=9 | energy=-5.2848298 | time=0.519s
iter=10 | energy=-5.2848298 | time=0.615s


-5.2848297789078025

In [19]:
excited_energies, excited_times, excited_MPS = DMRG(
    Hamiltonian = Hamiltonian,
    NKeep = NKeep,
    NSweep = NSweep,
    Krylov_bases = Krylov_bases,
    Lanczos_cutoff = Lanczos_cutoff,
    iterative_diag = False,
    orthogonal_to_list_of_MPS=[ground_MPS],
    two_site = two_site,
    verbose = verbose,
)

eigvals[1]

L=10 | NKeep=20 | NSweep=10 | iterative=False | two_site=True | Krylov_bases=5 | Lanczos_cutoff=0.01
iter=0 | energy=-0.01717453 | time=0.0s
iter=1 | energy=-4.7507212 | time=0.668s
iter=2 | energy=-4.7513389 | time=1.02s
iter=3 | energy=-4.7513389 | time=1.34s
iter=4 | energy=-4.7513389 | time=1.64s
iter=5 | energy=-4.7513389 | time=1.76s
iter=6 | energy=-4.7513389 | time=1.84s
iter=7 | energy=-4.7513389 | time=1.95s
iter=8 | energy=-4.7513389 | time=2.1s
iter=9 | energy=-4.7513389 | time=2.18s
iter=10 | energy=-4.7513389 | time=2.37s


-4.751502980728418

In [20]:
get_Neumann_entropy(ground_MPS)

array([0.11924839, 0.12692302, 0.1279663 , 0.1281325 , 0.12815813,
       0.12813472, 0.12797017, 0.12694886, 0.11927737])

In [1]:
get_Neumann_entropy(excited_MPS)

NameError: name 'get_Neumann_entropy' is not defined